# Atmospheric pressure correction

This notebook uses example data to generate an atmospheric correction file.

## Imports
Note you may need to uncomment the commented lines and put the path to stglib on your machine.

We set the directory where the example files are set, and read the atmospheric pressure data into an xarray Datset.

In [ ]:
from __future__ import division, print_function
# import sys
# sys.path.append('/Users/dnowacki/Documents/stglib')
import stglib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

basedir = '../examples/'
gndcrmet = xr.open_dataset(basedir + 'gndcrmet.nc')

## Function definitions

In [ ]:
def load_raw(site, dep, basedir, waves=False):
    fildir = basedir

    if waves == False:
        print(fildir + site + '1' + dep.upper() + 'aqd-raw.cdf')
        RAW = xr.open_dataset(fildir + site + '1' + dep.upper() + 'aqd-raw.cdf', autoclose=True)
    elif waves == True:
        print(fildir + site + '1' + dep.upper() + 'aqdwvs-raw.cdf')
        RAW = xr.open_dataset(fildir + site + '1' + dep.upper() + 'aqdwvs-raw.cdf', autoclose=True)
    
    return RAW, fildir

def proc_atmpres(site, dep, offset, basedir, waves=False):
    RAW, fildir = load_raw(site, dep, basedir, waves=waves)

    # take our atmos data and convert to decibars
    met = gndcrmet['BP']/100
    met = met.rename('atmpres')
    # reindex the met data onto the Aquadopp time base
    met = met.reindex(time=RAW['time'], copy=True, method='nearest')
    # set the atmospheric offset as an attribute
    met.attrs.update(offset=offset)
    
    if waves == False:
        print('in proc nowaves')
        met.to_netcdf(fildir + 'atmpres.cdf')
    elif waves == True:
        print('in proc waves')
        met.to_netcdf(fildir + 'atmpres-wvs.cdf')

    return RAW 

def load_clean(site, dep, basedir):
    fildir = basedir

    print(fildir + site + '1' + dep.upper() + 'aqd-a.nc')
    RAW = xr.open_dataset(fildir + site + '1' + dep.upper() + 'aqd-a.nc', decode_times=False, autoclose=True)
    RAW['time'] = RAW['time_cf']
    RAW = RAW.drop('time2')
    
    return xr.decode_cf(RAW)

## Generate the atmpres.cdf file 
This generates the file and embeds the instrument-specific offset as an attr. The trickiest part of this process is determining what to use as an offset. After you run this cell, you will have your very own atmpres.cdf file!

In [ ]:
# set offset for 1076b & 1078b to 10.15 to equalize postitive, negative anomalies for 
# atmos-corrected pressure at beginning/end of deployment
print('Processing Aquadopp currents')
site = '1076'
dep = 'a'
offset = -10.15

proc_atmpres(site, dep, offset, basedir, waves=False)

# print('Processing Aquadopp waves')
# for site, dep, offset in zip(['1076', '1076', '1078', '1078', '1079'], 
#                              ['a',    'b',    'a',    'b',    'b'], 
#                              [-10.15,  -10.25, -10.15, -10.25,  -9.95]):
    
#     proc_atmpres(site, dep, offset, basedir, waves=True)

## Load clean data
Note that you need to run the proper run scripts with your generated atmpres.cdf files... this only uses example files.

In [ ]:
RAW, fildir = load_raw(site, dep, basedir)
VEL = load_clean(site, dep, basedir)

## View data
See how the raw and P_1ac data compare.

In [ ]:
plt.figure(figsize=(10,8))
RAW['Pressure'].plot()
VEL['P_1ac'].plot()
plt.show()